In [1]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2
rng = np.random.default_rng(seed=42)

# Generating Delta Vector

In [2]:
def interpolation_for_u_h(nodal, mesh):
    """
    Takes in the solution of the forward solver at high resolution and returns the 
    corresponing values of u_h at a set xi that is predetermined 

    Returns: 
    Array of observation u_h that corresponds to each xi using interpolation if xi not 
    in final mesh. 
    """
    nodal = sl.assemble_nodal_values(nodal)
    mesh = np.array(mesh)  # Replace with your mesh node coordinates
    nodal = np.array(nodal).flatten()  # Replace with your computed u_h values   
    x_min = 0.0
    x_max = 1.0
    num_points = 100  # Number of xi's
    xi = np.linspace(x_min, x_max, num_points)

    u_h_interp = interp1d(mesh, nodal, kind='linear', fill_value="extrapolate")
    
    # Evaluate u_h at the observation points
    u_h_at_xi = u_h_interp(xi)

    return u_h_at_xi



In [9]:
def add_noise(observations_at_xi, num_points, sigma = 0.1):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    sigma = 0.1 # noise variance --> 0.01
    Sigma = (sigma ** 2) * np.eye(num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), Sigma)
    delta = observations_at_xi + noise 
    return delta

beta_true = np.array([0.5,1/6])
mesh_true, c_sol_true = sl.refinement_loop(0.00001, beta_true)
g_beta_true = interpolation_for_u_h(c_sol_true, mesh_true)
delta = add_noise(g_beta_true, num_points=100)
delta

array([-0.10600627, -0.09587289,  0.05285204,  0.17945516,  0.19082118,
        0.19309401,  0.44183219,  0.41898783,  0.48853577,  0.38477126,
        0.61574329,  0.64020868,  0.45819268,  0.7810412 ,  0.64481442,
        0.71333139,  0.76543723,  0.68307452,  0.82589351,  1.00593877,
        0.92158856,  1.10628169,  0.92252049,  0.96128832,  0.94787307,
        0.90423508,  0.93793643,  1.04308238,  1.44104229,  1.17357111,
        1.3037644 ,  1.16189238,  1.24948879,  1.39772785,  1.33145828,
        1.5374494 ,  1.42034488,  1.4751425 ,  1.3854539 ,  1.42764606,
        1.53977417,  1.36515757,  1.59581215,  1.38289805,  1.39352333,
        1.88349095,  1.37615556,  1.53995378,  1.56880043,  1.54579679,
        1.38444775,  1.46388122,  1.38644059,  1.32214736,  1.40895121,
        1.44101266,  1.47855968,  1.58320045,  1.42180022,  1.37268399,
        1.39739426,  1.38032867,  1.42898195,  1.41175083,  1.47088244,
        1.50439257,  1.33333876,  1.26400211,  1.37623318,  1.31

# Define Likelihood Function 

In [11]:
def phi(obseravations, predicted, sigma = 0.1) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    misfit = np.sum((obseravations - predicted) ** 2)
    return np.exp(-np.sum(misfit**2) / (2 * sigma**2))

# 2 cases, 1. denominator very close to 0 or both is 0. need to check if its one or both. 

phi(delta, g_beta_true, 0.1)

np.float64(2.336862087138976e-26)

# MCMC algorithm 

In [16]:
def MCMC(observations, number_of_iter, burn_in): 
    '''
    Builds a Markov Chain 

    Key Steps:
    1. Initialise a choice of Beta, beta_0 
    2. Compute likelihood of beta_0, using delta and beta_0_predicted
    3. Initialise the loop.
        - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
        - compute y_i and g(beta_i)
        - compute likelihood using {y_i and g(beta_i)}
        - set alpha = min{1, likelihood }
    '''
    # range of uniform distribution 
    x_star_range = (0.15, 0.85)
    r_range = (0, 0.15)

    # intiailise markov chain 
    chain = [] 
    beta_init = np.array([0.5, 1/4])
    beta_current = beta_init.copy()
    iter_count = 0
    acceptance_count = 0 
    acceptance_prob_history = []

    # initialise current observations and likelihood 
    mesh_current, c_sol_current = sl.refinement_loop(0.00001, beta = beta_current)
    y_current = interpolation_for_u_h(c_sol_current, mesh_current)
    likelihood_current = phi(observations, y_current)

    for i in range(number_of_iter):
        beta_proposal = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
        mesh_proposal, c_sol_proposal = sl.refinement_loop(0.00001, beta = beta_proposal)
        y_proposal = interpolation_for_u_h(c_sol_proposal, mesh_proposal)
        likelihood_proposal = phi(observations, y_proposal)

        # compute acceptance probability 
        acceptance_prob = min(1, likelihood_proposal/likelihood_current)
        acceptance_prob_history.append(acceptance_prob)
        print(acceptance_prob)
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_prob:
            beta_current = beta_proposal
            likelihood_current = likelihood_proposal
            acceptance_count += 1
        
        # Append the current beta to the chain
        chain.append(beta_current.copy())
        iter_count += 1
        print(iter_count)
    
    chain = np.array(chain)
    # # Discard burn-in and compute the MCMC estimate as the mean of the samples
    beta_mcmc = np.mean(chain[burn_in:], axis=0)
    return chain, beta_mcmc, acceptance_prob_history, acceptance_count

In [17]:
chain, beta_mcmc, acceptance_probablity_list, acceptance_count = MCMC(observations = delta, number_of_iter = 5000, burn_in=1000)
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmc)
print("Acceptance Probability History:", acceptance_probablity_list)
print("Acceptance Count:", acceptance_count)

0.0
1


KeyboardInterrupt: 